In [1]:
# Comp Econ Project 3 
!pip install ortools
from ortools.linear_solver import pywraplp
import pandas as pd 
import numpy as np

     |████████████████████████████████| 14.5 MB 24.3 MB/s 
     |████████████████████████████████| 126 kB 45.4 MB/s 
     |████████████████████████████████| 1.1 MB 38.4 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.12.0
    Uninstalling absl-py-0.12.0:
      Successfully uninstalled absl-py-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.4.0 requires absl-py<0.13,>=0.9, but you have absl-py 1.0.0 which is incompatible.


In [11]:
# Read in the data into a dataframe
all_df = pd.read_csv('fantasy_football_data.csv')

#Set parameters 
max_roster=15
max_budget=200

# Set the Roster limit as given to plugin the constraints
position_names=['DEF', 'K', 'QB', 'RB', 'TE', 'WR']
main_team = {'QB': 1,'WR': 3,'RB': 2,'TE': 1,'K': 1,'DEF': 1 }
roster_limit = {'QB': 2,'WR': 9,'RB': 8,'TE': 2,'K': 1,'DEF': 1 }

# Convert the dataframe columns to numpy arrays as we need them later 
points=all_df['ProjectedPoints'].to_numpy()
price=all_df['Price'].to_numpy()
position=all_df['Position'].to_numpy()

solver = pywraplp.Solver.CreateSolver('SCIP')

all_array= (all_df.to_numpy())

#####################
##### VARIABLE ######
#####################

# Iniate the variable, our solution is a binary variable for each player at each possible depth 
xis = np.array([np.array([solver.IntVar(0,1, str(all_array[i][0])+"-"+str(j+1) ) for j in range(roster_limit[all_array[i][1]])]) for i in range(len(all_array))])


#####################
#### CONSTRAINTS ####
#####################

# Each person occurs only once, sum over their depths 
for i in range(len(xis)):
  solver.Add(sum(xis[i]) <= 1)

# Maximum Roster limit is set to 15 
solver.Add(sum([sum(xis[i]) for i in range(len(xis))]) <= max_roster) 

# 200 Budget constraint is set
solver.Add(sum([sum(xis[i])*price[i] for i in range(len(xis))]) <= max_budget) 

# Each position has one at a particular depth
posn_sum={}
for posn in position_names:
  posn_sum[posn]=[]
  for j in range(roster_limit[posn]):
    posn_sum[posn].append(0)

for i in range(len(xis)):
  for j in range(roster_limit[position[i]]):
    posn_sum[position[i]][j]+=xis[i][j]

for posn in position_names:
  for j in range(roster_limit[posn]):
    solver.Add(posn_sum[posn][j]<=1) 
   
 
# First in each position is at least 1
posn_sum={}
for posn in position_names:
  posn_sum[posn]=0
for i in range(len(xis)):
  posn_sum[position[i]]+=xis[i][0]

for iter in posn_sum.values():
  solver.Add(iter>=1)       # Each position has one in first

for posn in posn_sum.keys():
  solver.Add(posn_sum[posn]<=main_team[posn]) # Each position less than main team for first

# Maximum Limit for each position to setup the Bench 
posn_sum={}
for posn in position_names:
  posn_sum[posn]=0
for i in range(len(xis)):
  for j in range(len(xis[i])):
    posn_sum[position[i]]+=xis[i][j]

for posn in posn_sum.keys():
  solver.Add(posn_sum[posn]<=roster_limit[posn]) # Each position less than overall limit


#####################
##### OBJECTIVE #####
#####################

sum_values=[]
for i in range(len(xis)):
  value=[]
  for dep in range(roster_limit[position[i]]):
    if dep+1<= main_team[position[i]]: # If the Main Team is lower
      val=xis[i][dep]*points[i]
    else:
      val=(1/(2**(dep+1-main_team[position[i]])))*xis[i][dep]*points[i]
    value.append(val)
  sum_values.append(sum(value))

solver.Maximize(sum(sum_values))

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
  print( ' The Integer Program has a Solution!')
  print('The projected points are ', solver.Objective().Value())
else:
    print('The problem does not have an optimal solution.')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


 The Integer Program has a Solution!
The projected points are  1754.2500000000005


In [25]:
selected=0
names=[]
poss=[]
depth=[]
types=[]
for i in range(len(xis)):
  k=0
  for j in xis[i]:
    selected+=j.solution_value()
    k+=1
    if j.solution_value()==1:
      names.append(all_array[i][0])
      poss.append(all_array[i][1])
      depth.append(str(j)[-1])
      if k <= main_team[position[i]]:
        types.append('Main Team')
      else:
        types.append('Bench')
    

df1=pd.DataFrame()
df1['Name']=names
df1['Depth']=depth
df1['Position']=poss
df1['Type of Player']=types
merged=df1.merge(all_df, how='left', left_on=['Name','Position'], right_on=['Name','Position'],indicator=True)

final_df=merged.sort_values(by='Type of Player',ascending=False).reset_index().drop(columns=['index', '_merge'])

final_df.to_csv('selected_players.csv')

print('The projected points are ', solver.Objective().Value())

print('The budget spent is ', final_df['Price'].sum())

print('The Players selected are the following, the csv is saved in the directory.')

final_df

The projected points are  1754.2500000000005
The budget spent is  200.0
The Players selected are the following, the csv is saved in the directory.


,Name,Depth,Position,Type of Player,Team,ProjectedPoints,Price
0,Cam Newton,1,QB,Main Team,CAR,278.8,16.6
1,Ty Montgomery,1,RB,Main Team,GBP,140.9,14.9
2,C.J. Anderson,2,RB,Main Team,DEN,128.8,7.9
3,Antonio Brown,3,WR,Main Team,PIT,194.6,55.3
4,Julio Jones,1,WR,Main Team,ATL,187.8,51.3
5,Emmanuel Sanders,2,WR,Main Team,DEN,125.7,15.2
6,Justin Tucker,1,K,Main Team,BAL,147.0,9.2
7,Martellus Bennett,1,TE,Main Team,GBP,95.9,6.9
8,Denver Broncos,1,DEF,Main Team,DEN,129.0,7.6
9,Tyrod Taylor,2,QB,Bench,BUF,255.8,5.1
